# 채팅 형식

이번 강의에서는 나만을 위한 챗봇을 생성하고 이용하는 방법에 대해 배웁니다.  
상황과 목적에 맞는 대화를 할 수 있는 챗봇을 만들어 봅시다.

## Setup

In [ ]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

get_completion 함수는 이전과 마찬가지로 하나의 프롬프트에 대해 답변하는 방식이죠.  

하지만 get_completion_from_messages는 여러 개의 메세지로 이루어진 입력이 주어지면,  
그 내용들을 바탕으로 답변하도록 합니다.  
우리가 ChatGPT에서 대화하던 그 방식이죠! 🤖

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]

예시는 다음 내용을 뜻합니다.
system : 언어 모델의 컨셉을 정해줍니다.  
user : 프롬프트를 나타냅니다.  
assistant : 언어 모델의 답변을 보여줍니다.  

순서상 assistant가 답변을 생성할 차례겠네요.

In [ ]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'}  ]

In [ ]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

In [ ]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Hi, my name is Isa'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

In [ ]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

In [ ]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

# 주문봇
<u>유저의 프롬프트와 모델의 답변을 자동으로 수집</u>하는 주문 봇을 만들 수 있습니다.  
아래 피자 가게에서 주문을 받는 언어 모델의 예시입니다. 

In [ ]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

함수 내의 inp는 아래에서 정의됩니다.  
유저가 내용을 직접 입력할 수 있도록 하는 기능을 포함하는 변수입니다.  
즉, 유저의 입력을 프롬프트로 받고, 다시 빈 칸으로 초기화하고 있는 것이죠.  
또한 마크다운 형식으로 답변하게 정해주었네요.

In [ ]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

context에는 가게에서 판매하는 피자와 관련된 정보가 담겨 있습니다.  
그리고 언어 모델이 고객을 어떻게 응대할지도 안내하고 있죠.  
조건이 모두 충족되기 전까지 언어 모델은 사용자에게 계속해서 질문할 것입니다.

In [ ]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},    
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},    

response = get_completion_from_messages(messages, temperature=0)
print(response)

위 코드도 자세히보면 편의를 위해 주문 정보를 json 형식으로 반환하도록 세팅한 것을 알 수 있습니다.  
이전에 답변 형식을 지정하거나 요청 사항을 순서대로 제시하는 것이 고품질의 답변을 얻어내는 데 유용하다고 했던 것을 떠올려보세요!

## 스스로 해보세요!
메뉴나 지시 사항을 수정해서 나만의 주문봇을 만들어보세요!
